In [1]:
import pandas as pd
import numpy as np
import re

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
import os
os.chdir('/Users/alvarodda/zrive-ds-oliva/zrive-ds-4q23-olive')

In [2]:
df = pd.read_parquet("notebooks/subset_muestreos_parcelas.parquet")
estados = [
    "estado_fenologico_1",
    "estado_fenologico_2",
    "estado_fenologico_3",
    "estado_fenologico_4",
    "estado_fenologico_5",
    "estado_fenologico_6",
    "estado_fenologico_7",
    "estado_fenologico_8",
    "estado_fenologico_9",
    "estado_fenologico_10",
    "estado_fenologico_11",
    "estado_fenologico_12",
    "estado_fenologico_13",
    "estado_fenologico_14"]
def find_estado_with_value_two(row) -> int:
    """
    Returns numero del siguiente etado (yt+1). (e.g., 13)
    """
    for column in row.index:
        if row[column] == 2:
            number_growth_stage = int(column.split("_")[-1])
            return number_growth_stage

def get_valid_dataset(df: pd.DataFrame(), max_days_till_next_date: int) -> pd.DataFrame():
    
    # Removing null and filling empty estados with 0
    df = df.dropna(subset=estados, how="all")
    df = df.dropna(subset=["codparcela"])
    df[estados] = df[estados].fillna(0)

    #Filtering Dataset to keep rows only with one unique 2
    df["count_2s"] = df[estados].eq(2).sum(axis=1)
    df = df[df.count_2s == 1]

    # Sorting by date
    df["fecha"] = pd.to_datetime(df["fecha"])
    df.sort_values(by="fecha", inplace=True)

    # Creating a column to display the number of days till the next observation
    df["next_date"] = df.groupby("codparcela", observed=True)["fecha"].shift(-1)
    df["days_until_next_visit"] = (df["next_date"] - df["fecha"]).dt.days

    # Creating a column to display the next estado_fenológico (yt+1)
    df["estado_actual"] = df[estados].apply(find_estado_with_value_two, axis=1)
    df["next_estado"] = df.groupby("codparcela", observed=True)["estado_actual"].shift(-1)

    # Removing the parcels with only one entry and the last entry for every parcel
    df = df.dropna(subset=["days_until_next_visit"])  # 5150 entries removed

    #Removing entries where estado decreases
    mask_estado_decrease = df["next_estado"] - df["estado_actual"] < 0
    df = df[~mask_estado_decrease]

    #Changing datatypes
    df["days_until_next_visit"] = df["days_until_next_visit"].astype("int16")
    df["next_estado"] = df["next_estado"].astype("int8")
    df[estados] = df[estados].astype("int8")

    # Filtering the max days
    df = df[df["days_until_next_visit"] < max_days_till_next_date]

    excluded_columns = ['count_2s', 'next_date']

    return df.loc[:, [i for i in df.columns if i not in excluded_columns]]


df =get_valid_dataset(df,32)
df

,generated_muestreos,codparcela,provincia,municipio,fecha,campaña,poligono,parcela,recinto,subrecinto,estado_fenologico_1,estado_fenologico_2,estado_fenologico_3,estado_fenologico_4,estado_fenologico_5,estado_fenologico_6,estado_fenologico_7,estado_fenologico_8,estado_fenologico_9,estado_fenologico_10,estado_fenologico_11,estado_fenologico_12,estado_fenologico_13,estado_fenologico_14,porcentaje_floracion,generated_parcelas,102_coordenada_x_(utm),103_coordenada_y_(utm),104_altitud_(m),105_comarca,201_superf_cultivada_en_la_parcela_agrícola_(ha),202_pendiente_(%),203_orientación,204_textura_del_suelo,206_secano_/_regadío,211_utilización_de_cubierta_vegetal,212_tipo_de_cubierta_vegetal,214_cultivo_asociado/otro_aprovechamiento,301_marco_(m_x_m),302_densidad_(plantas/ha),303_nº_pies_por_árbol,304_formación,305_diámetro_de_copa_(m),308_variedad_principal,311_fecha_de_plantación_variedad_principal,208_riego:_procedencia_del_agua,209_riego:_calidad_del_agua,313_variedad_secundaria,107_zona_homogénea,120_zona_biológica_raif,401_estación_climática_asociada,402_sensor_climático_asociado,207_riego:_sistema_usual_de_riego,108_u_h_c_a_la_que_pertenece,316_fecha_de_plantación_variedad_secundaria,315_patrón_variedad_secundaria,317_%_superficie_ocupada_variedad_secundaria,306_altura_de_copa_(m),310_patrón_variedad_principal,411_representa_a_la_u_h_c_(si/no),109_sistema_para_el_cumplimiento_gestión_integrada,lat,lon,days_until_next_visit,estado_actual,next_estado
388249,2020-04-23 17:00:15,001-00163-00-00,malaga,archidona,2005-08-30,2005,1,163,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,NaN,2020-04-23 14:19:57,369846.0,4110714.0,680.0,NORTE O ANTEQUERA,23.870001,6.0,NO,FRANCO-ARCILLOSO,SECANO,SI,SILVESTRE FORMADA CON HERBICIDA,NO,13 X 13,59.0,3.0,TRADICIONAL,7.0,HOJIBLANCO,10/10/1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.133744,-4.465325,7,10,10
374542,2020-04-23 17:00:15,001-00162-00-00,malaga,antequera,2005-08-30,2005,1,162,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,NaN,2020-04-23 14:19:57,369846.0,4110714.0,680.0,NORTE O ANTEQUERA,23.870001,6.0,NO,FRANCO-ARCILLOSO,SECANO,SI,SILVESTRE FORMADA CON HERBICIDA,NO,13 X 13,59.0,3.0,TRADICIONAL,7.0,HOJIBLANCO,10/10/1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.133744,-4.465325,7,10,10
385818,2020-04-23 17:00:15,128-00013-00-00,malaga,antequera,2005-08-31,2005,128,13,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,NaN,2020-04-23 14:19:57,362725.0,4117041.0,500.0,NORTE O ANTEQUERA,42.209999,3.0,SE,FRANCO-ARCILLO-ARENOSO,REGADÍO,NO,NaN,NO,12 X 12,69.0,69.0,TRADICIONAL,7.5,HOJIBLANCO,05/02/1900,POZO,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.189738,-4.546631,6,10,10
386907,2020-04-23 17:00:15,166-00500-00-00,malaga,antequera,2005-08-31,2005,166,500,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,NaN,2020-04-23 14:19:57,357448.0,4095675.0,640.0,NORTE O ANTEQUERA,35.000000,10.0,SO,FRANCO-ARCILLOSO,RIEGO DE APOYO,SI,SILVESTRE FORMADA CON HERBICIDA,NaN,7 X 7,204.0,1.0,GLOBO,2.5,PICUAL / MARTEÑO,05/09/1997,BALSA,APTAS PARA EL RIEGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.996423,-4.602006,5,10,10
385226,2020-04-23 17:00:15,110-00004-00-00,malaga,antequera,2005-08-31,2005,110,4,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,NaN,2020-04-23 14:19:57,352841.0,4102826.0,480.0,NORTE O ANTEQUERA,99.940002,2.0,S,FRANCO-ARCILLO-ARENOSO,SECANO,SI,SILVESTRE FORMADA CON HERBICIDA,NaN,12 X 12,69.0,3.0,TRADICIONAL,8.0,HOJIBLANCO,12/02/1901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.060148,-4.655158,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538714,2021-04-05 17:26:06,004-00023-01-01,sevilla,badolatosa,2021-03-24,2021,4,23,1,1,0,1,2,0,0,0,0,0,0,0,0,0,0,0,NaN,2021-04-05 17:24:32,351718.0,4125641.0,NaN,ESTEPA,0.570000,NaN,NaN,NaN,Secano,Si,Silvestre,

In [3]:
df_parcelas = df.groupby(by='codparcela').first()
df_parcelas

,generated_muestreos,provincia,municipio,fecha,campaña,poligono,parcela,recinto,subrecinto,estado_fenologico_1,estado_fenologico_2,estado_fenologico_3,estado_fenologico_4,estado_fenologico_5,estado_fenologico_6,estado_fenologico_7,estado_fenologico_8,estado_fenologico_9,estado_fenologico_10,estado_fenologico_11,estado_fenologico_12,estado_fenologico_13,estado_fenologico_14,porcentaje_floracion,generated_parcelas,102_coordenada_x_(utm),103_coordenada_y_(utm),104_altitud_(m),105_comarca,201_superf_cultivada_en_la_parcela_agrícola_(ha),202_pendiente_(%),203_orientación,204_textura_del_suelo,206_secano_/_regadío,211_utilización_de_cubierta_vegetal,212_tipo_de_cubierta_vegetal,214_cultivo_asociado/otro_aprovechamiento,301_marco_(m_x_m),302_densidad_(plantas/ha),303_nº_pies_por_árbol,304_formación,305_diámetro_de_copa_(m),308_variedad_principal,311_fecha_de_plantación_variedad_principal,208_riego:_procedencia_del_agua,209_riego:_calidad_del_agua,313_variedad_secundaria,107_zona_homogénea,120_zona_biológica_raif,401_estación_climática_asociada,402_sensor_climático_asociado,207_riego:_sistema_usual_de_riego,108_u_h_c_a_la_que_pertenece,316_fecha_de_plantación_variedad_secundaria,315_patrón_variedad_secundaria,317_%_superficie_ocupada_variedad_secundaria,306_altura_de_copa_(m),310_patrón_variedad_principal,411_representa_a_la_u_h_c_(si/no),109_sistema_para_el_cumplimiento_gestión_integrada,lat,lon,days_until_next_visit,estado_actual,next_estado
codparcela,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
000-00900-00-U0,2020-04-23 14:39:25,granada,alfacar,2006-04-24,2006,0,900,0,<NA>,1,2,1,0,0,0,0,0,0,0,0,0,0,0,70.0,2020-04-23 14:19:57,449521.0,4124050.0,975.0,DE LA VEGA,2.650,2.0,NO,FRANCO-ARENOSO,SECANO,NO,NaN,NaN,8 X 8,156.0,3.0,VASO,NaN,PICUAL / MARTEÑO,01/03/1968,NaN,NaN,HOJIBLANCO,GR/OL/02,GR/OL/02 LA VEGA,GR008,GR008,NaN,NaN,01/03/1968,NINGUNO,NaN,NaN,NINGUNO,NaN,NaN,37.261645,-3.569286,9,2,2
000-00999-00-00,2020-04-23 14:39:25,granada,nivar,2006-05-02,2006,0,999,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,90.0,2020-04-23 14:19:57,449772.0,4123208.0,975.0,DE LA VEGA,0.800,3.0,N,FRANCO-ARENOSO,SECANO,NO,NaN,NaN,7x7,210.0,3.0,VASO,NaN,HOJIBLANCO,01/03/1957,NaN,NaN,NaN,GR/OL/02,GR/OL/02 LA VEGA,GR008,GR008,NaN,NaN,NaN,NaN,NaN,NaN,HOJIBLANCA,NaN,NaN,37.254069,-3.566399,7,2,3
000-00999-00-FC,2020-04-23 14:39:25,granada,nivar,2006-05-02,2006,0,999,0,<NA>,0,2,1,0,0,0,0,0,0,0,0,0,0,0,NaN,2020-04-23 14:19:57,449772.0,4123208.0,975.0,DE LA VEGA,0.800,3.0,N,FRANCO-ARENOSO,SECANO,NO,NaN,NaN,7x7,210.0,3.0,VASO,NaN,HOJIBLANCO,01/03/1957,NaN,NaN,NaN,GR/OL/02,GR/OL/02 LA VEGA,GR008,GR008,NaN,NaN,NaN,NaN,NaN,NaN,HOJIBLANCA,NaN,NaN,37.254069,-3.566399,7,2,3
001-00001-01,2021-04-05 17:26:06,jaen,carboneros,2017-08-22,2017,1,1,1,<NA>,0,0,0,0,0,0,0,0,0,2,0,0,0,0,NaN,2021-04-05 17:24:32,444376.0,4228325.0,NaN,SIERRA MORENA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JA/OL/01 SIERRA MORENA,NaN,NaN,NaN,CARBONEROS,NaN,NaN,NaN,NaN,NaN,NaN,Producción Integrada (PI),38.201158,-3.635286,7,10,10
001-00002-00-00,2020-04-23 14:34:36,cordoba,palma del rio,2006-03-22,2006,1,2,0,0,1,2,1,0,0,0,0,0,0,0,0,0,0,0,NaN,2020-04-23 14:19:57,297056.0,4179284.0,60.0,CAMPIÑA BAJA,3.000,NaN,NaN,NaN,REGADÍO,SI,SILVESTRE FORMADA CON HERBICIDA,NaN,7 X 4,357.0,1.0,NaN,NaN,HOJIBLANCO,14/05/1996,RÍO,APTAS PARA EL RIEGO,NaN,CO/OL/05,CO/OL/05 LAS COLONIAS-VEGA BAJA,CO009,NaN,NaN,LAS VEREDAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.738364,-5.303189,7,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421-00097-01,2020-04-23 16:54:05,jaen,villacarrillo,2014-03-24,2014,421,97,1,<NA>,2,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,2020-04-23 14:19:57,501108.0,4212845.0,NaN,LA LOMA,1.140,NaN,NaN,NaN,Secano,NaN,NaN,NaN,10x10,100.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,J

In [4]:
binary_features = [
    '211_utilización_de_cubierta_vegetal',
    '214_cultivo_asociado/otro_aprovechamiento',
    '208_riego:_procedencia_del_agua',
    '209_riego:_calidad_del_agua',
    '207_riego:_sistema_usual_de_riego',
    '109_sistema_para_el_cumplimiento_gestión_integrada',
    'porcentaje_floracion'
]

def replace_nullwithmean_remove_outliers(df, threshold = np.inf):
    df = df.copy()
    mean_value = df.mean()
    df.fillna(mean_value, inplace=True)
    df.loc[df > threshold] = mean_value
    df.astype('int32')
    return df
    

df_parcelas = df_parcelas[binary_features]
df_parcelas.isnull().sum()/len(df_parcelas) * 100

211_utilización_de_cubierta_vegetal                   25.764757
214_cultivo_asociado/otro_aprovechamiento             88.776389
208_riego:_procedencia_del_agua                       74.924601
209_riego:_calidad_del_agua                           78.457561
207_riego:_sistema_usual_de_riego                     65.855235
109_sistema_para_el_cumplimiento_gestión_integrada    64.971995
porcentaje_floracion                                  20.228350
dtype: float64

In [5]:
df_parcelas.isnull().sum(axis=1).value_counts()

5    1171
4     985
6     870
3     486
2     439
1     341
7     251
0      99
Name: count, dtype: int64

In [6]:
replace_dict = {'Si': 1, 'No': 0, 'NO': 0,'SI':1}
df_parcelas["211_utilización_de_cubierta_vegetal"] = df_parcelas["211_utilización_de_cubierta_vegetal"].replace(replace_dict).fillna(0)

/var/folders/dz/f2v0bzjj2451zsrm0t4gx04m0000gn/T/ipykernel_12005/3779340218.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parcelas["211_utilización_de_cubierta_vegetal"] = df_parcelas["211_utilización_de_cubierta_vegetal"].replace(replace_dict).fillna(0)


In [7]:
df_parcelas["211_utilización_de_cubierta_vegetal"].value_counts()

211_utilización_de_cubierta_vegetal
0    2685
1    1957
Name: count, dtype: int64

In [1]:
df_parcelas['214_cultivo_asociado/otro_aprovechamiento'].value_counts()

NameError: name 'df_parcelas' is not defined

In [9]:
replace_dict_riego = {'Pozo': 1, 'POZO': 1}
def apply_dict(row, replace_dict):
    if row in replace_dict.keys():
        return replace_dict[row]
    else:
        return 0

df_parcelas['208_riego:_procedencia_del_agua'] = df_parcelas['208_riego:_procedencia_del_agua'].astype('str').apply(apply_dict, replace_dict=replace_dict_riego)
df_parcelas['208_riego:_procedencia_del_agua'].value_counts()

/var/folders/dz/f2v0bzjj2451zsrm0t4gx04m0000gn/T/ipykernel_12005/3185757845.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parcelas['208_riego:_procedencia_del_agua'] = df_parcelas['208_riego:_procedencia_del_agua'].astype('str').apply(apply_dict, replace_dict=replace_dict_riego)


208_riego:_procedencia_del_agua
0    3973
1     669
Name: count, dtype: int64

In [10]:
df_parcelas['209_riego:_calidad_del_agua'].value_counts()

209_riego:_calidad_del_agua
APTAS PARA EL RIEGO                                   441
Aptas para riego                                      431
UTILIZABLES CON PRECAUCIONES                           25
APTA PARA RIEGO                                        21
APTAS PARA RIEGO                                       20
APTA                                                   10
Utilizable con precauciones                             8
OPTIMA                                                  7
BUENA                                                   6
Aptas para riogo                                        5
No aptas para riego                                     5
SIN DEFINIR                                             4
REGULAR-VERTICILOSIS, SALINIDAD, PARTÍCULAS SUSPEN      4
MEDIANA CALIDAD                                         4
-                                                       2
BUENA- AGUA DE LLUVIA                                   2
BUENA-POTABLE                               

In [11]:
df_parcelas['207_riego:_sistema_usual_de_riego'].value_counts()



207_riego:_sistema_usual_de_riego
Localizado gotero                             816
LOCALIZADO GOTERO                             479
goteo                                          47
GOTEO                                          34
NINGUNO                                        32
LOCALIZADO MICROASPERSOR                       29
GRAVEDAD MANTA                                 17
LOCALIZADO LATIGUILLO                          14
Gravedad manta                                 12
LOCALIZADO POR GOTEO                            9
Localizado microaspersor                        9
Goteo                                           6
LOCALIZADO GOTERO SUPERFICIAL                   6
SUBTERRANEO                                     5
GRAVEDAD PIES                                   5
No existe sistema                               5
SECANO                                          5
RIEGO GOTEO                                     4
Gravedad pies                                   4
localizado tuber

In [12]:
df_parcelas['207_riego:_sistema_usual_de_riego'] = df_parcelas['207_riego:_sistema_usual_de_riego'].str.lower()

dict_replace = {}
for i in df_parcelas['207_riego:_sistema_usual_de_riego'].str.lower().unique():
    if pd.isna(i):
        continue
    if "got" in i:
        dict_replace[i] = 1
    else: dict_replace[i] = 0


df_parcelas['207_riego:_sistema_usual_de_riego'] = df_parcelas['207_riego:_sistema_usual_de_riego'].apply(apply_dict, replace_dict=dict_replace).fillna(0)
df_parcelas['207_riego:_sistema_usual_de_riego'].value_counts()

/var/folders/dz/f2v0bzjj2451zsrm0t4gx04m0000gn/T/ipykernel_12005/3287797602.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parcelas['207_riego:_sistema_usual_de_riego'] = df_parcelas['207_riego:_sistema_usual_de_riego'].str.lower()
/var/folders/dz/f2v0bzjj2451zsrm0t4gx04m0000gn/T/ipykernel_12005/3287797602.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parcelas['207_riego:_sistema_usual_de_riego'] = df_parcelas['207_riego:_sistema_usual_de_riego'].apply(apply_dict, replace_dict=dict_replace

207_riego:_sistema_usual_de_riego
0    3233
1    1409
Name: count, dtype: int64

In [13]:
replace_dict_riego = {'Producción Integrada (PI)': 1}
def apply_dict(row, replace_dict):
    if row in replace_dict.keys():
        return replace_dict[row]
    else:
        return 0

print(df_parcelas['109_sistema_para_el_cumplimiento_gestión_integrada'].value_counts())
df_parcelas['109_sistema_para_el_cumplimiento_gestión_integrada'] = df_parcelas['109_sistema_para_el_cumplimiento_gestión_integrada'].apply(apply_dict, replace_dict=replace_dict_riego).fillna(0)
df_parcelas['109_sistema_para_el_cumplimiento_gestión_integrada'].value_counts()

109_sistema_para_el_cumplimiento_gestión_integrada
Producción Integrada (PI)        1577
Entidad de Asesoramiento (EA)      37
Otro sistema Asesorado (OA)        11
Agricultura Ecológica (AE)          1
Producción certificada (PC)         0
Name: count, dtype: int64


/var/folders/dz/f2v0bzjj2451zsrm0t4gx04m0000gn/T/ipykernel_12005/3223546872.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parcelas['109_sistema_para_el_cumplimiento_gestión_integrada'] = df_parcelas['109_sistema_para_el_cumplimiento_gestión_integrada'].apply(apply_dict, replace_dict=replace_dict_riego).fillna(0)


109_sistema_para_el_cumplimiento_gestión_integrada
0.0    3065
1.0    1577
Name: count, dtype: int64

In [14]:
df_parcelas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4642 entries, 000-00900-00-U0 to 504-10003-03-1
Data columns (total 7 columns):
 #   Column                                              Non-Null Count  Dtype   
---  ------                                              --------------  -----   
 0   211_utilización_de_cubierta_vegetal                 4642 non-null   category
 1   214_cultivo_asociado/otro_aprovechamiento           521 non-null    category
 2   208_riego:_procedencia_del_agua                     4642 non-null   int64   
 3   209_riego:_calidad_del_agua                         1000 non-null   category
 4   207_riego:_sistema_usual_de_riego                   4642 non-null   int64   
 5   109_sistema_para_el_cumplimiento_gestión_integrada  4642 non-null   float64 
 6   porcentaje_floracion                                3703 non-null   float32 
dtypes: category(3), float32(1), float64(1), int64(2)
memory usage: 178.3+ KB


In [15]:
df_parcelas.describe()

,208_riego:_procedencia_del_agua,207_riego:_sistema_usual_de_riego,109_sistema_para_el_cumplimiento_gestión_integrada,porcentaje_floracion
count,4642.000000,4642.000000,4642.000000,3703.000000
mean,0.144119,0.303533,0.339724,34.323170
std,0.351248,0.459833,0.473667,32.756546
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,5.000000
50%,0.000000,0.000000,0.000000,21.000000
75%,0.000000,1.000000,1.000000,60.000000
max,1.000000,1.000000,1.000000,100.000000


In [16]:
print(df_parcelas['211_utilización_de_cubierta_vegetal'].dtype)

category


In [17]:
df.groupby(by='codparcela')['porcentaje_floracion'].mean()


codparcela
000-00900-00-U0    70.000000
000-00999-00-00    90.000000
000-00999-00-FC          NaN
001-00001-01             NaN
001-00002-00-00          NaN
                     ...    
421-00097-01             NaN
501-00340-02-00    41.833332
501-00343-01-00    24.166666
501-00400-01-00    38.000000
504-10003-03-1     81.500000
Name: porcentaje_floracion, Length: 4642, dtype: float32